<a href="https://colab.research.google.com/github/pdpavandesai/MergingDiabetesData/blob/main/Trained_Model_For_Diabetes_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import warnings
from numpy.lib.polynomial import poly
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

In [ ]:
# Uploading symptomatic featured data and clinical test data

from google.colab import files
uploaded = files.upload()

Saving symptomatic_dataset.csv to symptomatic_dataset.csv
Saving clinical_test_dataset.csv to clinical_test_dataset.csv


In [ ]:
# Reading symptomatic featured data and clinical test data
symptomatic_data = pd.read_csv('symptomatic_dataset.csv')
clinical_test_data = pd.read_csv('clinical_test_dataset.csv')

In [ ]:
# Creating a copy of above data
sd_copy = symptomatic_data.copy()
ctd_copy = clinical_test_data.copy()

In [ ]:
# Splitting symptomatic_data into training & testing data
X_train_sd, X_test_sd, y_train_sd, y_test_sd = train_test_split(sd_copy.drop('Diabetes', axis=1), sd_copy['Diabetes'], test_size=0.25, random_state=42)

In [ ]:
# Splitting clinical_test_data into training & testing data
X_train_ctd, X_test_ctd, y_train_ctd, y_test_ctd = train_test_split(ctd_copy.drop('Diabetes', axis=1), ctd_copy['Diabetes'], test_size=0.25, random_state=42)

In [ ]:
# Training Decision Tree classifier for symptomatic dataset
X_train_sd, X_test_sd, y_train_sd, y_test_sd = train_test_split(sd_copy.drop('Diabetes', axis=1), sd_copy['Diabetes'], test_size=0.25, random_state=42)
model_sd = DecisionTreeClassifier()
model_sd.fit(X_train_sd, y_train_sd)

DecisionTreeClassifier()

In [ ]:
# Training Multi-Layer Perceptron classifier for clinical feature dataset
X_train_ctd, X_test_ctd, y_train_ctd, y_test_ctd = train_test_split(ctd_copy.drop('Diabetes', axis=1), ctd_copy['Diabetes'], test_size=0.25, random_state=42)
model_ctd = MLPClassifier(hidden_layer_sizes=(128, 64, 32), activation='logistic', solver='lbfgs', max_iter= 2000)
model_ctd.fit(X_train_ctd, y_train_ctd)

MLPClassifier(activation='logistic', hidden_layer_sizes=(128, 64, 32),
              max_iter=2000, solver='lbfgs')

In [ ]:
def patientSymptomaticToDiabetes(polyuria, polydipsia, suddenWeightLoss, partialParesis, polyphagia, visualBlurring, irritability):
  patient_symptoms = pd.DataFrame({'Polyuria': [polyuria], 'Polydipsia': [polydipsia], 'SuddenWeightLoss': [suddenWeightLoss], 'PartialParesis': [partialParesis]
                             ,'Polyphagia': [polyphagia],'Irritability':[irritability],'VisualBlurring':[visualBlurring]})
  symptoms_based_prediction = model_sd.predict(patient_symptoms)
  if symptoms_based_prediction == 1:
    print('Recommended below medical tests:\n')
    print('1. Blood sugar test\n')
    print('2. Hb1Aclevel\n')
    print('3. Blood pressure test\n')
    print('4. Cholestrol test\n')
    print('5. BMI\n')
  else:
    print('Recommended to consult a doctor as the symptoms are not leading to diabetes\n')

  return symptoms_based_prediction

In [ ]:
def confirmingDiabetesAndProvidingRecommendation(age, bmi, haveHighBP, haveHighCholestrol, blood_glucose_level, Hb1Aclevel, patient):
  patient_clinical_information = pd.DataFrame({'Age': [age], 'BMI': [bmi], 'HighChol': [haveHighBP], 'HighBP': [haveHighCholestrol]})
  clinical_diabetic_confirmation = model_ctd.predict(patient_clinical_information)
  if clinical_diabetic_confirmation == 1:
    print('Based on your age & clinical data, we conclude and recommend the following:\n')
    if((age>=4 or age<=6) or (age>=9 or age<=11)):
      if(blood_glucose_level >= 100 and blood_glucose_level <=120):
        print('Dear ',patient,' your blood sugar level indicates you are pre-diabetic and we recommend you the following')
      else:
        print('Dear ',patient,' your blood sugar level indicates you are diabetic and we recommend you the following')
    else:
      print('')
  else:
    print('Recommended to consult a doctor as the symptoms are not leading to diabetes\n')

  return clinical_diabetic_confirmation

In [ ]:
def categorizeAge(age):
  if age>=18 and age<=24:
    return 1
  elif age>=25 and age<=29:
    return 2
  elif age>=30 and age<=34:
    return 3
  elif age>=35 and age<=39:
    return 4
  elif age>=40 and age<=44:
    return 5
  elif age>=45 and age<=49:
    return 6
  elif age>=50 and age<=54:
    return 7
  elif age>=55 and age<=59:
    return 8
  elif age>=60 and age<=64:
    return 9
  elif age>=65 and age<=69:
    return 10
  elif age>= 70 and age<=74:
    return 11
  elif age>=75 and age<=79:
    return 12
  elif age>=80:
    return 13
  else:
    return 0

In [ ]:
def consultApp():
  polyuria = input('Do you have polyuria?\n')
  if polyuria.lower() == 'yes':
    polyuria = 1
  else:
    polyuria = 0
  polydipsia = input('Do you have polydipsia?\n')
  if polydipsia.lower() == 'yes':
    polydipsia = 1
  else:
    polydipsia = 0
  irritability = input('Do you have irritability?\n')
  if irritability.lower() == 'yes':
    irritability = 1
  else:
    irritability = 0
  visualBlurring = input('Do you have visual blurring?\n')
  if visualBlurring.lower() == 'yes':
    visualBlurring = 1
  else:
    visualBlurring = 0
  polyphagia = input('Do you have polyphagia?\n')
  if polyphagia.lower() == 'yes':
    polyphagia = 1
  else:
    polyphagia = 0
  suddenWeightLoss = input('Do you have sudden weight loss?\n')
  if suddenWeightLoss.lower() == 'yes':
    suddenWeightLoss = 1
  else:
    suddenWeightLoss = 0
  partialParesis = input('Do you have partial paresis?\n')
  if partialParesis.lower() == 'yes':
    partialParesis = 1
  else:
    partialParesis = 0

  age = int(input('Enter your age\n'))
  age = categorizeAge(age)
  name = input('Enter your name\n')

  isSymptomaticToDiabetes = patientSymptomaticToDiabetes(polyuria, polydipsia, suddenWeightLoss, partialParesis, polyphagia, visualBlurring, irritability)
  if isSymptomaticToDiabetes == 1:
    bmi = float(input('Enter your BMI\n'))
    haveHighBP = input('Do you have high blood pressure?\n')
    if haveHighBP.lower() == 'yes':
      haveHighBP = 1
    else:
      haveHighBP = 0
    haveHighCholestrol = input('Do you have high cholestrol?\n')
    if haveHighCholestrol.lower() == 'yes':
      haveHighCholestrol = 1
    else:
      haveHighCholestrol = 0
    fastingSugarLevel = float(input('Enter your fasting sugar level\n'))
    postSugarLevel = float(input('Enter your post sugar level\n'))
    Hb1Aclevel = float(input('Enter your Hb1Ac level\n'))

    confirmingDiabetesAndProvidingRecommendation(age, bmi, haveHighBP, haveHighCholestrol, fastingSugarLevel, postSugarLevel, Hb1Aclevel, name)

In [ ]:
consultApp()

Do you have polyuria?
yes
Do you have polydipsia?
no
Do you have irritability?
no
Do you have visual blurring?
yes
Do you have polyphagia?
yes
Do you have sudden weight loss?
no
Do you have partial paresis?
no
Enter your age
34
Enter your name
Desai
Recommended below medical tests:

1. Blood sugar test

2. Hb1Aclevel

3. Blood pressure test

4. Cholestrol test

5. BMI

Enter your BMI
20
Do you have high blood pressure?
no
Do you have high cholestrol?
no
Enter your fasting sugar level
120
Enter your post sugar level
200
Enter your Hb1Ac level
12
Recommended to consult a doctor as the symptoms are not leading to diabetes

